#### Project Targets:
- Summary of Top 5 Alert Codes (September 2021)
- Develop data-driven solutions to mitigate the issue of high manual queue checks
- Validation using data from OCT/21 to Jan/22

#### PART ONE: CREATE THE CONNECTION TO SNOWFLAKE AND USE SNOWPARK

In [ ]:
# import packages

from snowflake.snowpark.functions import *
from snowflake.snowpark import Session
import pandas as pd

In [ ]:
>>> connection_parameters = {
...    "account": "<your snowflake account>",
...    "user": "<your snowflake user>",
...    "password": "<your snowflake password>",
...    "role": "<your snowflake role>",  # optional
...    "warehouse": "<your snowflake warehouse>",  # optional
...    "database": "<your snowflake database>",  # optional
...    "schema": "<your snowflake schema>",  # optional
...  }  

>>> session = Session.builder.configs(connection_parameters).create()

#### PART TWO: EXPLORT THE DATA - APPLICATIONS

In [ ]:
# get the master data

df_applications = session.table('DB_NAME.SCHEMA_NAME.applications')\
                         .select('*')

applications_df.show()

In [ ]:
# only pick up Sep/21
df_applications_sep = df_applications.filter(
                                              (col('submit_dt') >= '2021-09-01')
                                            & (col('submit_dt') < '2021-10-01')
                                            )

In [ ]:
# assign a hardward flag to the applications
df_applications_1 = df_applications_sep.withColumn('hardware_fg', when(col('product_name')).isNotNull(), 'Y').otherwise('N')

# Only look at the hardware applications
df_applications_1_final = df_applications_1.filter(col('hardware_fg') == 'Y')

#### PART THREE: GET THE VISA INFO

In [ ]:
# Get the visa infomation from table Passport 

df_passport = session.table('DB_NAME.SCHEMA_NAME.Passport')\
                     .select('*')
df_passport.show()

In [ ]:
# dealing with the missing values of the cd_type_visa and in the passport dataset
# rename cd_type_visa to visa_status
df_passport_1 = df_passport.withColumn('visa_status', when(col('cd_type_visa')).isNotNull(), col('cd_type_visa')).otherwise('N/A')

# drop cd_type_visa
df_passport_2 = df_passport_1.drop('cd_type_visa')

In [ ]:
# select visa_status from passport to join master data 
df_passport_3 = df_passport_2.select([
                                    col('uuid').name('b_uuid'),
                                    col('submit_dt').name('b_submit_dt')
                                     'visa_status'
                                     ])

df_apps_visa = df_applications_1_final.join(df_passport_3, 
                                                        (df_applications_1_final.uuid == df_passport_3.b_uuid) &
                                                        (df_applications_1_final.submit_dt == df_passport_3.b_submit_dt), 
                                                        'left'
                                                        )\
                                      .drop(['b_uuid', 'b_submit_dt'])\
                                      .distinct()\
                                      .sort(col('submit_dt').asc())

df_apps_visa.show()

#### PART THREE: GET THE ALERT INFO

In [ ]:
# Get the alert info
df_alert = session.table('DB_NAME.SCHEMA_NAME.alert')\
                  .select('*')
df_alert.show()

In [ ]:
# Only looking at 'Decline-Pending', 'Refer', 'BiometricsRequired'
df_alert_1 = df_alert.filter(col('status')._in(['Decline-Pending', 'Refer', 'BiometricsRequired']))

In [ ]:
# select alert_code to join into master data 
df_alert_2 = df_alert_1.select([
                                    col('uuid').name('b_uuid'),
                                    col('submit_dt').name('b_submit_dt')
                                     'alert_code'
                                     ])

df_apps_visa_alert = df_apps_visa.join(df_alert_2, 
                                                        (df_apps_visa.uuid == df_alert_2.b_uuid) &
                                                        (df_apps_visa.submit_dt == df_alert_2.b_submit_dt), 
                                                        'left'
                                                        )\
                                      .drop(['b_uuid', 'b_submit_dt'])\
                                      .distinct()\
                                      .sort(col('submit_dt').asc())
df_apps_visa_alert.show()

In [ ]:
# get the alert_code_desc based on the code_desc introduction
ddf_apps_visa_alert_1 = f_apps_visa_alert.withColumn('alert_code_desc', when(col('alert_code').isin(['D18','D181']), 'D - Scorecard Related'),
                                                                        when(col('alert_code').isin(["D2", "D32", "D7"]), 'D - Debt related'),
                                                                        when(col('alert_code').isin(["D20", "D99"]), 'D - Eligibility'),
                                                                        when(col('alert_code') == "D222", 'D - Collections related'),
                                                                        when(col('alert_code') == "D300", 'D - Iovation related'),
                                                                        when(col('alert_code').isin(["D43", "D44", "D30"]), 'D - Business eligibility'),
                                                                        when(col('alert_code').isin(["D55", "D56"]), 'D - Business debt related')
                                                    ).otherwise(None)

#### PART THREE: GET THE QUEUES INFO

In [ ]:
# Get the Queues info
df_queues = session.table('DB_NAME.SCHEMA_NAME.queues_info')\
                  .select('*')
df_queues.show()

In [ ]:
# only pick up queue_id
df_queues_1 = df_queues.select([
                                    col('uuid').name('b_uuid'),
                                    col('submit_dt').name('b_submit_dt')
                                     'QUEUE_ID'
                                     ])

In [ ]:
# join into master data 
df_sum_final = df_apps_visa_alert_1.join(df_queues_1, 
                                                        (df_apps_visa_alert_1.uuid == df_queues_1.b_uuid) &
                                                        (df_apps_visa_alert_1.submit_dt == df_queues_1.b_submit_dt), 
                                                        'left'
                                                        )\
                                      .drop(['b_uuid', 'b_submit_dt'])\
                                      .distinct()\
                                      .sort(col('submit_dt').asc())

In [ ]:
# Divided order into manual queue and automatic queue with a flag 
df_sum_final_1 = df_sum_final.withColumn('queue_flag', when(col('queuue_id').isin(['BUREAU ERROR QUEUE DEALER',
                                                                                   'BUREAU ERROR QUEUE NONDEALER',
                                                                                   'CUSTOMER ENRICHMENT QUEUE',
                                                                                   'FINAL REFER QUEUE DEALER',
                                                                                   'FINAL REFER QUEUE NONDEALER',
                                                                                   'NO ID QUEUE','ONLINE AUTOMATION QUEUE',
                                                                                   'BUREAU ERROR QUEUE',
                                                                                   'IDENTIFICATION ERROR QUEUE'
                                                                                   ]), 'Manual Queue'
                                                            ).otherwise('Automation Queue'))



In [ ]:
# Output as a csv folder stored in local using pandas 
df_sum = df_sum_final_1.to_pandas()

df_sum.to_csv('file_name.csv')